In [1]:
%cd ..
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import librosa
import numpy as np

from population import Population
from sample_library import SampleLibrary


c:\Users\justi\coding\Uni\MA


In [4]:
X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)
print(clf.predict([[0, 0, 0, 0]]))

[1]


### Population to feature vector of shape (n_features)

"The following statistics are proposed as approximative features.  
For each instrument and each onset in the approximated
music track, we save the smallest distance between the best
candidate mixture which contains this instrument and the onset
to approximate.  
(pop.best_collections_per_onset)  
  
These smallest distances are kept during the
complete evolutionary loop in an archive and do not represent
the final population only. (TODO!) 
    
Then, we estimate the mean, the
minimum, and the maximum values for each of 51 instrument
and 88 theoretically possible pitches for two different analysis
frames of 10s and 3s. Additionally, we sort the recognised
instruments based on the smallest distances, and assign ranks
to corresponding approximative features, e.g., value of “rang
of acoustic guitar” = 1 means that acoustic guitar had the
smallest mean distance between approximations with this
instrument and unknown onsets in the analysis frame. This
leads to an overall number of feature dimensions equal to
(51 · 3 + 88 · 3 + 51) · 2 = 936."

In [2]:
pop = Population.from_file("test.pkl")
target, sr = librosa.load(librosa.ex('nutcracker'), duration=30)
lib = SampleLibrary()

Starting Thread-0
Starting Thread-1
Starting Thread-2
Starting Thread-3
Starting Thread-4
Starting Thread-5
Starting Thread-6
Starting Thread-7
2: Progress: 0
1: Progress: 0
3: Progress: 0
4: Progress: 0
0: Progress: 0
6: Progress: 0
7: Progress: 0
5: Progress: 0
2: Progress: 50
4: Progress: 50
7: Progress: 50
1: Progress: 50
0: Progress: 50
3: Progress: 50
6: Progress: 50
5: Progress: 50
2: Progress: 100
7: Progress: 100
4: Progress: 100
1: Progress: 100
6: Progress: 100
3: Progress: 100
0: Progress: 100
5: Progress: 100
2: Progress: 150
4: Progress: 150
7: Progress: 150
1: Progress: 150
6: Progress: 150
0: Progress: 150
3: Progress: 150
5: Progress: 150
2: Progress: 200
4: Progress: 200
7: Progress: 200
1: Progress: 200
6: Progress: 200
0: Progress: 200
3: Progress: 200
5: Progress: 200
2: Progress: 250
7: Progress: 250
1: Progress: 250
4: Progress: 250
6: Progress: 250
0: Progress: 250
5: Progress: 250
3: Progress: 250


In [26]:
## Choose two different analysis frames of 10s and 3s
end_offset = 10 * sr
possible_onsets = len(target) - end_offset
window_start = np.random.randint(low=0, high=possible_onsets)
window_end = window_start + end_offset

## Grab the onsets in those windows, and the associated best records from the population
relevant_collections = [collection for collection in pop.best_collections_per_onset.values() if collection.onset in range(window_start, window_end)]

## For each window separately, calculate the maximum, minimum, and mean fitnesses for each occasion of
## An instrument
instrument_occurrences_fitness = dict()
## A pitch
pitch_occurrences_fitness = dict()

for collection in relevant_collections:
    for sample in collection.samples:
        if sample.instrument in instrument_occurrences_fitness:
            instrument_occurrences_fitness[sample.instrument].append(collection.fitness)
        else:
            instrument_occurrences_fitness[sample.instrument] = [collection.fitness]
        if sample.pitch in pitch_occurrences_fitness:
            pitch_occurrences_fitness[sample.pitch].append(collection.fitness)
        else:
            pitch_occurrences_fitness[sample.pitch] = [collection.fitness]

# print(instrument_occurrences_fitness)
# print(pitch_occurrences_fitness)

instrument_min = {instrument: np.min(instrument_occurrences_fitness[instrument]) for instrument in instrument_occurrences_fitness}
instrument_max = {instrument: np.max(instrument_occurrences_fitness[instrument]) for instrument in instrument_occurrences_fitness}
instrument_mean = {instrument: np.mean(instrument_occurrences_fitness[instrument]) for instrument in instrument_occurrences_fitness}

pitch_min = {pitch: np.min(pitch_occurrences_fitness[pitch]) for pitch in pitch_occurrences_fitness}
pitch_max = {pitch: np.max(pitch_occurrences_fitness[pitch]) for pitch in pitch_occurrences_fitness}
pitch_mean = {pitch: np.mean(pitch_occurrences_fitness[pitch]) for pitch in pitch_occurrences_fitness}

## Finally, give each instrument a rank from 1 to n_instruments, based on their mean distances (smallest = rank 1, highest = rank n_instruments)
# instrument_sort = np.argsort([instrument_mean[instrument] for instrument in instrument_mean])
instrument_sort = {k: v for k, v in sorted(instrument_mean.items(), key=lambda item: item[1])}
instrument_ranks = {instrument: i + 1 for i, instrument in enumerate(instrument_sort)}

# Create feature vector
instrument_features = []
for instrument_info in lib.instruments:
    instr_name = instrument_info.name
    if instr_name in instrument_ranks:
        instrument_features.append([instrument_min[instr_name], instrument_mean[instr_name], instrument_max[instr_name], instrument_ranks[instr_name]])
    else:
        instrument_features.append([np.inf, np.inf, np.inf, np.inf])
pitch_features = []
for pitch in lib.pitches:
    if pitch in pitch_min:
        pitch_features.append([pitch_min[pitch], pitch_mean[pitch], pitch_max[pitch]])
    else:
        pitch_features.append([np.inf, np.inf, np.inf])
flat_instr_features = np.array(instrument_features).flatten()
flat_pitch_features = np.array(pitch_features).flatten()
features = np.concatenate((flat_instr_features, flat_pitch_features))
print(f"{len(features)} generated from {len(lib.instruments)} instruments and {len(lib.pitches)} pitches.")


197 generated from 8 instruments and 55 pitches.


19
63
